<a href="https://colab.research.google.com/github/caileymm/cuda-by-example-exercises/blob/main/chapter_3_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.11.13
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp3p3756kn".


In [3]:
%%writefile book.h
/*
 * Copyright 1993-2010 NVIDIA Corporation.  All rights reserved.
 *
 * NVIDIA Corporation and its licensors retain all intellectual property and
 * proprietary rights in and to this software and related documentation.
 * Any use, reproduction, disclosure, or distribution of this software
 * and related documentation without an express license agreement from
 * NVIDIA Corporation is strictly prohibited.
 *
 * Please refer to the applicable NVIDIA end user license agreement (EULA)
 * associated with this source code for terms and conditions that govern
 * your use of this NVIDIA software.
 *
 */


#ifndef __BOOK_H__
#define __BOOK_H__
#include <stdio.h>

static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))


#define HANDLE_NULL( a ) {if (a == NULL) { \
                            printf( "Host memory failed in %s at line %d\n", \
                                    __FILE__, __LINE__ ); \
                            exit( EXIT_FAILURE );}}

template< typename T >
void swap( T& a, T& b ) {
    T t = a;
    a = b;
    b = t;
}


void* big_random_block( int size ) {
    unsigned char *data = (unsigned char*)malloc( size );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}

int* big_random_block_int( int size ) {
    int *data = (int*)malloc( size * sizeof(int) );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}


// a place for common kernels - starts here

__device__ unsigned char value( float n1, float n2, int hue ) {
    if (hue > 360)      hue -= 360;
    else if (hue < 0)   hue += 360;

    if (hue < 60)
        return (unsigned char)(255 * (n1 + (n2-n1)*hue/60));
    if (hue < 180)
        return (unsigned char)(255 * n2);
    if (hue < 240)
        return (unsigned char)(255 * (n1 + (n2-n1)*(240-hue)/60));
    return (unsigned char)(255 * n1);
}

__global__ void float_to_color( unsigned char *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset*4 + 0] = value( m1, m2, h+120 );
    optr[offset*4 + 1] = value( m1, m2, h );
    optr[offset*4 + 2] = value( m1, m2, h -120 );
    optr[offset*4 + 3] = 255;
}

__global__ void float_to_color( uchar4 *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset].x = value( m1, m2, h+120 );
    optr[offset].y = value( m1, m2, h );
    optr[offset].z = value( m1, m2, h -120 );
    optr[offset].w = 255;
}


#if _WIN32
    //Windows threads.
    #include <windows.h>

    typedef HANDLE CUTThread;
    typedef unsigned (WINAPI *CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC unsigned WINAPI
    #define  CUT_THREADEND return 0

#else
    //POSIX threads.
    #include <pthread.h>

    typedef pthread_t CUTThread;
    typedef void *(*CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC void
    #define  CUT_THREADEND
#endif

//Create thread.
CUTThread start_thread( CUT_THREADROUTINE, void *data );

//Wait for thread to finish.
void end_thread( CUTThread thread );

//Destroy thread.
void destroy_thread( CUTThread thread );

//Wait for multiple threads.
void wait_for_threads( const CUTThread *threads, int num );

#if _WIN32
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void *data){
        return CreateThread(NULL, 0, (LPTHREAD_START_ROUTINE)func, data, 0, NULL);
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        WaitForSingleObject(thread, INFINITE);
        CloseHandle(thread);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        TerminateThread(thread, 0);
        CloseHandle(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        WaitForMultipleObjects(num, threads, true, INFINITE);

        for(int i = 0; i < num; i++)
            CloseHandle(threads[i]);
    }

#else
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void * data){
        pthread_t thread;
        pthread_create(&thread, NULL, func, data);
        return thread;
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        pthread_join(thread, NULL);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        pthread_cancel(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        for(int i = 0; i < num; i++)
            end_thread( threads[i] );
    }

#endif




#endif  // __BOOK_H__

Writing book.h


In [ ]:
%%writefile hello_world.cu
// 3.2.1 Hello, World!

#include <stdio.h>
#include "book.h"

int main( void ) {
  printf( "Hello, World!\n" );
  return 0;
}

Overwriting hello_world.cu


In [ ]:
!nvcc hello_world.cu -o hello_world
!./hello_world

Hello, World!


In [ ]:
%%writefile a_kernel_call.cu
// 3.2.2 A Kernel Call

#include <stdio.h>
#include "book.h"

__global__ void kernel(void) {}

int main(void) {
    kernel<<<1,1>>>();
    printf("Hello, World!\n");
    return 0;
}

// NOTES:
// - __global__ alerts the complier that a function should be complied to run on
//   a device instead of the host
// - <<1, 1>> = arguments we plan to pass to the runtime system

Writing a_kernel_call.cu


In [ ]:
!nvcc a_kernel_call.cu -o a_kernel_call
!./a_kernel_call

Hello, World!


In [ ]:
%%writefile passing_parameters.cu
// 3.2.3 Passing Parameters

#include <stdio.h>
#include "book.h"

// CUDA kernel function defined using __global__
__global__ void add(int a, int b, int *c) {
    *c = a + b;
}

// host (CPU) code
int main(void) {
    int c;
    int *dev_c; // points to memory on the device (GPU)

    // allocates memory on the device (GPU)
    HANDLE_ERROR(cudaMalloc((void**)&dev_c, sizeof(int)));

    add<<<1, 1>>>(2, 7, dev_c); // <<<1, 1>>> means one block with one thread

    // copies result from the device memory pointed to by dev_c back to the
    // host memory location of the variable c
    HANDLE_ERROR(cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost));

    printf("2 + 7 = %d\n", c);
    cudaFree(dev_c); // deallocates the memory on the device (GPU)

    return 0;
}

// NOTES:
// - cudaMalloc() tells the CUDA runtime to allocate the memory on the device.
// - The first argument is a pointer to the pointer you want to hold the address
//   of the newly allocated memory, and the second parameter is the size of the
//   allocation you want to make.
// - It returns a pointer (a memory address) that refers to a location on the
//   device.
// - RULES:
//    - You can pass pointers allocated with cudaMalloc() to functions that
//      execute on the device.
//    - You can use pointers allocated with cudaMalloc()to read or write
//      memory from code that executes on the device.
//    - You can pass pointers allocated with cudaMalloc()to functions that
//      execute on the host.
//    - You cannot use pointers allocated with cudaMalloc()to read or write
//      memory from code that executes on the host.
// - CPU's memory controller cannot directly access the GPU's memory (!!)

Writing passing_parameters.cu


In [ ]:
!nvcc passing_parameters.cu -o passing_parameters
!./passing_parameters

2 + 7 = 0


In [ ]:
%%writefile querying_devices.cu
// 3.3 Querying Devices

#include <stdio.h>
#include "book.h"

int main(void) {
  cudaDeviceProp prop;

  int count;
  HANDLE_ERROR(cudaGetDeviceCount(&count));
  for (int i = 0; i < count; i++) {
    HANDLE_ERROR(cudaGetDeviceProperties(&prop, i));
    printf(" --- General Information for device %d ---\n", i);
    printf("Name: %s\n", prop.name);
    printf("Compute capability: %d.%d\n", prop.major, prop.minor);
    printf("Clock rate: %d\n", prop.clockRate);
    printf("Device copy overlap: ");
    if (prop.deviceOverlap)
        printf("Enabled\n");
    else
        printf("Disabled\n");
    printf("Kernel execition timeout : ");
    if (prop.kernelExecTimeoutEnabled)
        printf("Enabled\n");
    else
        printf("Disabled\n");
    printf(" --- Memory Information for device %d ---\n", i);
    printf("Total global mem: %ld\n", prop.totalGlobalMem);
    printf("Total constant Mem: %ld\n", prop.totalConstMem);
    printf("Max mem pitch: %ld\n", prop.memPitch);
    printf("Texture Alignment: %ld\n", prop.textureAlignment);
    printf(" --- MP Information for device %d ---\n", i);
    printf("Multiprocessor count: %d\n", prop.multiProcessorCount);
    printf("Shared mem per mp: %ld\n", prop.sharedMemPerBlock);
    printf("Registers per mp: %d\n", prop.regsPerBlock);
    printf("Threads in warp: %d\n", prop.warpSize);
    printf("Max threads per block: %d\n", prop.maxThreadsPerBlock);
    printf("Max thread dimensions: (%d, %d, %d)\n", prop.maxThreadsDim[0],
            prop.maxThreadsDim[1],prop.maxThreadsDim[2]);
    printf("Max grid dimensions: (%d, %d, %d)\n", prop.maxGridSize[0],
            prop.maxGridSize[1], prop.maxGridSize[2]);
    printf("\n");
  }
}

Writing querying_devices.cu


In [ ]:
!nvcc querying_devices.cu -o querying_devices
!./querying_devices

 --- General Information for device 0 ---
Name: Tesla T4
Compute capability: 7.5
Clock rate: 1590000
Device copy overlap: Enabled
Kernel execition timeout : Disabled
 --- Memory Information for device 0 ---
Total global mem: 15828320256
Total constant Mem: 65536
Max mem pitch: 2147483647
Texture Alignment: 512
 --- MP Information for device 0 ---
Multiprocessor count: 40
Shared mem per mp: 49152
Registers per mp: 65536
Threads in warp: 32
Max threads per block: 1024
Max thread dimensions: (1024, 1024, 64)
Max grid dimensions: (2147483647, 65535, 65535)



In [10]:
%%writefile using_device_prop.cu

// 3.4 Using Device Properties

#include <stdio.h>
#include "book.h"

int main( void ) {
  cudaDeviceProp prop; // stores properies of a device
  int dev;

  HANDLE_ERROR( cudaGetDevice( &dev ) ); // stores id of current device in dev
  printf( "ID of current CUDA device: %d\n", dev );

  memset( &prop, 0, sizeof( cudaDeviceProp ) ); // initializes prop strucutre with 0's
  // looking for device with compute capability of 1.3
  prop.major = 1;
  prop.minor = 3;
  HANDLE_ERROR( cudaChooseDevice( &dev, &prop ) ); // searches for device that best matches properties
  printf( "ID of CUDA device closest to revision 1.3: %d\n", dev );
  HANDLE_ERROR( cudaSetDevice( dev ) ); // sets specified device for application
}

// NOTES:
// - Systems with multiple GUPs are becoming more common
// - If application depends on certain features of the GPU or depends on having
//   the fastest GPU in the system, this API is necessary
// - No guarantee that the CUDA runtime will choose the best or most appropriate
//   GPU for application

Overwriting using_device_prop.cu


In [6]:
!nvcc using_device_prop.cu -o using_device_prop
!./using_device_prop

ID of current CUDA device: 0
ID of CUDA device closest to revision 1.3: 0
